## Imports

In [26]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-10-03-11-55-41.csv')

In [27]:
gdf = gpd.read_file('Miami-Dade_Boundary.geojson')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['September']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    809
Name: count, dtype: int64
-------
$/SQUARE FEET
False    800
True       9
Name: count, dtype: int64
-------
YEAR BUILT
False    809
Name: count, dtype: int64
-------


In [7]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Bay-Harbor-Islands/9521-E-Bay-Harbor-Dr-33154/unit-502/home/112924631


In [8]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [9]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [10]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
116,PAST SALE,September-5-2023,Condo/Co-op,10801 SW 109th Ct Unit D110,Kendall,FL,33176.0,255.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10801-SW-109th...,MARMLS,A11414155,N,Y,25.670000,-80.371302
780,PAST SALE,September-19-2023,Condo/Co-op,1355 NE 167th St #210,North Miami Beach,FL,33162.0,91000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1355-NE-167th-...,Beaches MLS,F10349299,N,Y,25.929738,-80.173163
483,PAST SALE,September-26-2023,Condo/Co-op,245 NE 191st St #3007,Miami,FL,33179.0,100000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/245-NE-191st-S...,MARMLS,A11412415,N,Y,25.950534,-80.196455
125,PAST SALE,September-11-2023,Condo/Co-op,19000 NE 3rd Ct #412,Miami,FL,33179.0,100000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19000-NE-3rd-C...,MARMLS,A11419173,N,Y,25.949177,-80.194522
135,PAST SALE,September-29-2023,Condo/Co-op,11 NE 204th St #5,Miami Gardens,FL,33179.0,103600.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11-NE-204th-St...,Beaches MLS,F10387904,N,Y,25.962979,-80.198775
672,PAST SALE,September-29-2023,Condo/Co-op,1750 NE 191st St Unit 821-3,Miami,FL,33179.0,120000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1750-NE-191st-...,MARMLS,A11365745,N,Y,25.949213,-80.167412
480,PAST SALE,September-26-2023,Condo/Co-op,19000 NE 3rd Ct #420,Miami,FL,33179.0,120000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19000-NE-3rd-C...,MARMLS,A11413733,N,Y,25.949177,-80.194522
370,PAST SALE,September-12-2023,Condo/Co-op,18700 NE 3rd Ct #621,Miami,FL,33179.0,120000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18700-NE-3rd-C...,MARMLS,A11400857,N,Y,25.947261,-80.194442
697,PAST SALE,September-28-2023,Condo/Co-op,20330 NE 2nd Ave #8,Miami Gardens,FL,33179.0,125999.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20330-NE-2nd-A...,MARMLS,A11352940,N,Y,25.961636,-80.198534
740,PAST SALE,September-15-2023,Condo/Co-op,952 NE 199th St #114,Miami,FL,33179.0,130000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/95...,Beaches MLS,F10369584,N,Y,25.956937,-80.182803


In [16]:
print(df_filtered['URL'].iloc[116])

https://www.redfin.com/FL/Miami/10801-SW-109th-Ct-33176/unit-D110/home/43153383


In [13]:
# Correct the prices, if needed
df_filtered.at[116,'PRICE']=(225000)

In [14]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [17]:
# # Corrections, if needed
df_filtered.at[116,'$/SQUARE FEET']=(210000/930)

In [18]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
370,120000.0,18700 NE 3rd Ct #621,Miami,101.0
5,150000.0,900 NE 195th St #218,Miami,101.0
125,100000.0,19000 NE 3rd Ct #412,Miami,109.0
672,120000.0,1750 NE 191st St Unit 821-3,Miami,116.0
631,168000.0,1770 NE 191st St Unit 417-1,Miami,128.0
480,120000.0,19000 NE 3rd Ct #420,Miami,130.0
780,91000.0,1355 NE 167th St #210,North Miami Beach,132.0
557,149900.0,1770 NE 191st St Unit 600-1,Miami,141.0
720,275000.0,1800 NE 114th St #601,Miami,141.0
749,150000.0,1680 NE 191st St Unit 308-2,Miami,145.0


In [19]:
print(df_filtered.URL.iloc[370])

https://www.redfin.com/FL/Miami/18700-NE-3rd-Ct-33179/unit-621/home/43010107


In [107]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [20]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [21]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [22]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [23]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [24]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [29]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"August 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)
        
folium.GeoJson(gdf,tooltip='Miami-Dade County',name='Miami-Dade County').add_to(m)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [30]:
m.save('index.html')

## Summary Info

In [31]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [32]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [33]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [34]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/1355-NE-167th-St-33162/unit-210/home/43014072


In [35]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
CONTINUUM ON SOUTH BEACH, 100 S Pointe Dr #3303 Miami Beach | Price $15,150,000 | $4,543 psf | Year built: 2003
Least Expensive
Mar-len Gardens, 1355 NE 167th St #210 North Miami Beach | Price $91,000 | $132 psf | Year built: 1964
Highest Price Per Square Foot
CONTINUUM ON SOUTH BEACH, 100 S Pointe Dr #3303 Miami Beach | Price $15,150,000 | $4,543 psf | Year built: 2003
Lowest Price Per Square Foot
CHANTILLY CONDO, 900 NE 195th St #218 Miami | Price $150,000 | $101 psf | Year built: 1981
Newest
227 NE 2ND STREET CONDO, 227 NE 2nd St #2004 Miami | Price $485,000 | $942 psf | Year built: 2022
Oldest
THE FOUNTAIN CONDO HOTEL, 334 Euclid Ave #101 Miami Beach | Price $405,000 | $923 psf | Year built: 1924


## Time on Market Calculator

In [120]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [121]:
from datetime import datetime, timedelta

date1 = datetime(2023, 7, 7) ## List (Earlier) date
date2 = datetime(2023, 8, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

54


## Map URL Snagger

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [37]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_sep_2023


## Get Summary Data

In [38]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 809
--------
Total sale price: $548,483,879
--------
Median sale price: $397,000
--------
Max sale price: $15,150,000
--------
Min sale price: $91,000
------------------------------------------------
PSF INFO
Max price per square foot: $4,543
--------
Min price per square foot: $101
--------
Median price per square foot: $420
------------------------------------------------
CONDO AGES
Newest building: 2022.0
----------
Oldest building: 1924.0
----------
Average building age: 1987.841779975278
